In [2]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [3]:
class DatasetFromFolder(Dataset):
    def __init__(self, directory, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.directory = directory
        self.image_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.input_transform = input_transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert('RGB')

        if self.input_transform:
            input_img = self.input_transform(img)
        else:
            input_img = img

        if self.target_transform:
            target_img = self.target_transform(img)
        else:
            target_img = img

        return input_img, target_img

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 9, 1, 4),
            nn.PReLU(),
            nn.Conv2d(64, 128, 1),
            nn.PReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.PReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.PReLU(),
            nn.Conv2d(32, 3, 3, 1, 1),
            nn.Tanh()
        )
    def forward(self, x):
        return self.model(x)

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(512, 1)
        
    def forward(self, x):
        x = self.main(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return torch.sigmoid(x)

In [6]:
class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        self.mse_loss = nn.MSELoss()

    def forward(self, fake_out, fake_img, real_img):
        content_loss = self.mse_loss(fake_img, real_img)
        adversarial_loss = -torch.log(fake_out).mean()
        return content_loss + 0.001 * adversarial_loss

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
input_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

target_transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(), 
])

In [9]:
train_dataset = DatasetFromFolder(
    "F:\\super_resolution_project\\hr_crowd_img",
    input_transform=input_transform,
    target_transform=target_transform
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

generator = Generator().to(device)
discriminator = Discriminator().to(device)
g_optimizer = Adam(generator.parameters(), lr=0.002)
d_optimizer = Adam(discriminator.parameters(), lr=0.002)
g_loss_fn = GeneratorLoss().to(device)

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    for low_res, high_res in train_loader:
        low_res, high_res = low_res.to(device), high_res.to(device)
        
        discriminator.zero_grad()
        high_res_fake = generator(low_res).detach()
        real_output = discriminator(high_res)
        fake_output = discriminator(high_res_fake)
        d_loss_real = torch.log(real_output + 1e-8)
        d_loss_fake = torch.log(1.0 - fake_output + 1e-8)
        d_loss = -(d_loss_real + d_loss_fake).mean()
        d_loss.backward()
        d_optimizer.step()
        
        generator.zero_grad()
        high_res_fake = generator(low_res)
        fake_output = discriminator(high_res_fake)
        g_loss = g_loss_fn(fake_output, high_res_fake, high_res)
        g_loss.backward()
        g_optimizer.step()

        print(f"Epoch {epoch+1}: Gen Loss: {g_loss.item()}, Disc Loss: {d_loss.item()}")


Epoch 1: Gen Loss: 0.23996087908744812, Disc Loss: 1.3895670175552368
Epoch 1: Gen Loss: 0.20397140085697174, Disc Loss: 1.068406105041504
Epoch 1: Gen Loss: 0.10717155039310455, Disc Loss: 2.4975318908691406
Epoch 1: Gen Loss: 0.09716176241636276, Disc Loss: 2.097585916519165
Epoch 1: Gen Loss: 0.11783725768327713, Disc Loss: 7.143011093139648
Epoch 1: Gen Loss: 0.04529137164354324, Disc Loss: 2.8509466648101807
Epoch 1: Gen Loss: 0.06505467742681503, Disc Loss: 1.3964154720306396
Epoch 1: Gen Loss: 0.09880516678094864, Disc Loss: 1.6062090396881104
Epoch 1: Gen Loss: 0.06567578762769699, Disc Loss: 1.3493036031723022
Epoch 1: Gen Loss: 0.038935232907533646, Disc Loss: 1.6263517141342163
Epoch 1: Gen Loss: 0.052635613828897476, Disc Loss: 1.2158641815185547
Epoch 1: Gen Loss: 0.06322388350963593, Disc Loss: 1.4449719190597534
Epoch 1: Gen Loss: 0.04390982538461685, Disc Loss: 1.3835058212280273
Epoch 1: Gen Loss: 0.0425921231508255, Disc Loss: 1.4587526321411133
Epoch 1: Gen Loss: 0.0

Epoch 1: Gen Loss: 0.016306091099977493, Disc Loss: 1.419082760810852
Epoch 1: Gen Loss: 0.015333589166402817, Disc Loss: 1.4984935522079468
Epoch 1: Gen Loss: 0.01757434569299221, Disc Loss: 1.4804590940475464
Epoch 1: Gen Loss: 0.01618802919983864, Disc Loss: 1.4371387958526611
Epoch 1: Gen Loss: 0.012386083602905273, Disc Loss: 1.3942095041275024
Epoch 1: Gen Loss: 0.01408690121024847, Disc Loss: 1.3969295024871826
Epoch 1: Gen Loss: 0.013885618187487125, Disc Loss: 1.3833181858062744
Epoch 1: Gen Loss: 0.013858641497790813, Disc Loss: 1.3903440237045288
Epoch 1: Gen Loss: 0.013410007581114769, Disc Loss: 1.3976620435714722
Epoch 1: Gen Loss: 0.01744667813181877, Disc Loss: 1.3956480026245117
Epoch 1: Gen Loss: 0.01495969109237194, Disc Loss: 1.4042096138000488
Epoch 1: Gen Loss: 0.01327360887080431, Disc Loss: 1.3958029747009277
Epoch 1: Gen Loss: 0.012151757255196571, Disc Loss: 1.4045963287353516
Epoch 1: Gen Loss: 0.012989645823836327, Disc Loss: 1.3984849452972412
Epoch 1: Gen 

Epoch 1: Gen Loss: 0.011262843385338783, Disc Loss: 1.4089877605438232
Epoch 1: Gen Loss: 0.008911175653338432, Disc Loss: 1.386370301246643
Epoch 1: Gen Loss: 0.009990492835640907, Disc Loss: 1.3690718412399292
Epoch 1: Gen Loss: 0.01050163060426712, Disc Loss: 1.3869590759277344
Epoch 1: Gen Loss: 0.00906805694103241, Disc Loss: 1.3993109464645386
Epoch 1: Gen Loss: 0.009370054118335247, Disc Loss: 1.379826307296753
Epoch 1: Gen Loss: 0.009542650543153286, Disc Loss: 1.3673306703567505
Epoch 1: Gen Loss: 0.009999332949519157, Disc Loss: 1.3831851482391357
Epoch 1: Gen Loss: 0.010284451767802238, Disc Loss: 1.3942402601242065
Epoch 1: Gen Loss: 0.010836109519004822, Disc Loss: 1.376132607460022
Epoch 1: Gen Loss: 0.00937320850789547, Disc Loss: 1.3806650638580322
Epoch 1: Gen Loss: 0.00910398829728365, Disc Loss: 1.3867504596710205
Epoch 1: Gen Loss: 0.008033809252083302, Disc Loss: 1.3887839317321777
Epoch 1: Gen Loss: 0.008231702260673046, Disc Loss: 1.378395438194275
Epoch 1: Gen L

Epoch 2: Gen Loss: 0.009450558573007584, Disc Loss: 1.3718849420547485
Epoch 2: Gen Loss: 0.007164665963500738, Disc Loss: 1.3860056400299072
Epoch 2: Gen Loss: 0.007732645142823458, Disc Loss: 1.3942452669143677
Epoch 2: Gen Loss: 0.0077392589300870895, Disc Loss: 1.3887898921966553
Epoch 2: Gen Loss: 0.00896866712719202, Disc Loss: 1.3902690410614014
Epoch 2: Gen Loss: 0.009072882123291492, Disc Loss: 1.3705183267593384
Epoch 2: Gen Loss: 0.007846755906939507, Disc Loss: 1.4092020988464355
Epoch 2: Gen Loss: 0.007262678816914558, Disc Loss: 1.3950400352478027
Epoch 2: Gen Loss: 0.008787763305008411, Disc Loss: 1.3746405839920044
Epoch 2: Gen Loss: 0.00761262746527791, Disc Loss: 1.378839135169983
Epoch 2: Gen Loss: 0.009062198922038078, Disc Loss: 1.402242660522461
Epoch 2: Gen Loss: 0.007427117321640253, Disc Loss: 1.3806735277175903
Epoch 2: Gen Loss: 0.007867501117289066, Disc Loss: 1.380997657775879
Epoch 2: Gen Loss: 0.00764405308291316, Disc Loss: 1.3904352188110352
Epoch 2: Ge

Epoch 2: Gen Loss: 0.009186756797134876, Disc Loss: 1.4044591188430786
Epoch 2: Gen Loss: 0.007288798689842224, Disc Loss: 1.424421787261963
Epoch 2: Gen Loss: 0.008191057480871677, Disc Loss: 1.3736786842346191
Epoch 2: Gen Loss: 0.007834674790501595, Disc Loss: 1.3282546997070312
Epoch 2: Gen Loss: 0.007324833422899246, Disc Loss: 1.3899974822998047
Epoch 2: Gen Loss: 0.008156755939126015, Disc Loss: 1.4046225547790527
Epoch 2: Gen Loss: 0.008884158916771412, Disc Loss: 1.3299530744552612
Epoch 2: Gen Loss: 0.008409942500293255, Disc Loss: 1.3829708099365234
Epoch 2: Gen Loss: 0.007101666182279587, Disc Loss: 1.3471910953521729
Epoch 2: Gen Loss: 0.007869108580052853, Disc Loss: 1.368422269821167
Epoch 2: Gen Loss: 0.008060936816036701, Disc Loss: 1.3451664447784424
Epoch 2: Gen Loss: 0.007678780239075422, Disc Loss: 1.3567653894424438
Epoch 2: Gen Loss: 0.00852093007415533, Disc Loss: 1.3422901630401611
Epoch 2: Gen Loss: 0.00874983798712492, Disc Loss: 1.3908751010894775
Epoch 2: G

...

Epoch 9: Gen Loss: 0.007481321692466736, Disc Loss: 0.5223740935325623
Epoch 9: Gen Loss: 0.007244276814162731, Disc Loss: 0.5315312147140503
Epoch 9: Gen Loss: 0.008848162367939949, Disc Loss: 0.5400426387786865
Epoch 9: Gen Loss: 0.009403777308762074, Disc Loss: 0.41664713621139526
Epoch 9: Gen Loss: 0.009277316741645336, Disc Loss: 0.8024042844772339
Epoch 9: Gen Loss: 0.00895463116466999, Disc Loss: 0.7788389921188354
Epoch 9: Gen Loss: 0.009188355877995491, Disc Loss: 0.615985095500946
Epoch 9: Gen Loss: 0.009460600093007088, Disc Loss: 2.1924493312835693
Epoch 9: Gen Loss: 0.009955223649740219, Disc Loss: 1.7061915397644043
Epoch 9: Gen Loss: 0.011579075828194618, Disc Loss: 3.1905601024627686
Epoch 9: Gen Loss: 0.014361604116857052, Disc Loss: 3.547642469406128
Epoch 9: Gen Loss: 0.01283020619302988, Disc Loss: 2.9834508895874023
Epoch 9: Gen Loss: 0.010465474799275398, Disc Loss: 2.1024346351623535
Epoch 9: Gen Loss: 0.008735858835279942, Disc Loss: 0.7996577620506287
Epoch 9: 

Epoch 9: Gen Loss: 0.007326776161789894, Disc Loss: 0.9196267127990723
Epoch 9: Gen Loss: 0.0070520732551813126, Disc Loss: 0.9085108041763306
Epoch 9: Gen Loss: 0.006288857664912939, Disc Loss: 1.0186927318572998
Epoch 9: Gen Loss: 0.007585596293210983, Disc Loss: 0.734943151473999
Epoch 9: Gen Loss: 0.006400781217962503, Disc Loss: 0.8543870449066162
Epoch 9: Gen Loss: 0.0068575311452150345, Disc Loss: 0.8592092990875244
Epoch 9: Gen Loss: 0.007204052992165089, Disc Loss: 0.7925107479095459
Epoch 9: Gen Loss: 0.007181174587458372, Disc Loss: 0.8556264638900757
Epoch 9: Gen Loss: 0.008567181415855885, Disc Loss: 0.6887094974517822
Epoch 9: Gen Loss: 0.007126315031200647, Disc Loss: 0.7579151391983032
Epoch 9: Gen Loss: 0.0083940913900733, Disc Loss: 0.6188060641288757
Epoch 9: Gen Loss: 0.008265359327197075, Disc Loss: 0.6831018924713135
Epoch 9: Gen Loss: 0.008011135272681713, Disc Loss: 0.6140127182006836
Epoch 9: Gen Loss: 0.007255158387124538, Disc Loss: 0.7842599153518677
Epoch 9

Epoch 9: Gen Loss: 0.03540872409939766, Disc Loss: 4.961697578430176
Epoch 9: Gen Loss: 0.019149141386151314, Disc Loss: 1.9203529357910156
Epoch 9: Gen Loss: 0.011035059578716755, Disc Loss: 1.5083383321762085
Epoch 9: Gen Loss: 0.019255857914686203, Disc Loss: 2.9133198261260986
Epoch 9: Gen Loss: 0.013234269805252552, Disc Loss: 3.0064287185668945
Epoch 9: Gen Loss: 0.01283489353954792, Disc Loss: 1.8566819429397583
Epoch 9: Gen Loss: 0.01300071645528078, Disc Loss: 1.3580127954483032
Epoch 9: Gen Loss: 0.013067211955785751, Disc Loss: 2.505690574645996
Epoch 9: Gen Loss: 0.010785266757011414, Disc Loss: 1.8610100746154785
Epoch 9: Gen Loss: 0.010181427001953125, Disc Loss: 1.2625575065612793
Epoch 9: Gen Loss: 0.011207791976630688, Disc Loss: 1.5349891185760498
Epoch 9: Gen Loss: 0.011813229881227016, Disc Loss: 1.7232145071029663
Epoch 9: Gen Loss: 0.013305636122822762, Disc Loss: 1.5050954818725586
Epoch 9: Gen Loss: 0.03237289935350418, Disc Loss: 1.2705059051513672
Epoch 9: Gen

Epoch 10: Gen Loss: 0.007956468500196934, Disc Loss: 1.2199597358703613
Epoch 10: Gen Loss: 0.006347022484987974, Disc Loss: 1.206511378288269
Epoch 10: Gen Loss: 0.007580397184938192, Disc Loss: 1.1401405334472656
Epoch 10: Gen Loss: 0.00699706980958581, Disc Loss: 1.2098220586776733
Epoch 10: Gen Loss: 0.007359373848885298, Disc Loss: 1.1954600811004639
Epoch 10: Gen Loss: 0.008366839028894901, Disc Loss: 1.1460933685302734
Epoch 10: Gen Loss: 0.008430607616901398, Disc Loss: 1.0241676568984985
Epoch 10: Gen Loss: 0.007926399819552898, Disc Loss: 1.0147948265075684
Epoch 10: Gen Loss: 0.009681439027190208, Disc Loss: 0.8965306878089905
Epoch 10: Gen Loss: 0.0074022794142365456, Disc Loss: 1.1247440576553345
Epoch 10: Gen Loss: 0.007588569074869156, Disc Loss: 1.1041663885116577
Epoch 10: Gen Loss: 0.007451464422047138, Disc Loss: 1.0059727430343628
Epoch 10: Gen Loss: 0.00759609742090106, Disc Loss: 0.8246297240257263
Epoch 10: Gen Loss: 0.00691137695685029, Disc Loss: 0.794569551944

Epoch 10: Gen Loss: 0.007598753087222576, Disc Loss: 0.7331582307815552
Epoch 10: Gen Loss: 0.006826542317867279, Disc Loss: 0.6961807012557983
Epoch 10: Gen Loss: 0.006475420668721199, Disc Loss: 0.7145665287971497
Epoch 10: Gen Loss: 0.00761913787573576, Disc Loss: 0.5200010538101196
Epoch 10: Gen Loss: 0.008602526038885117, Disc Loss: 0.45674702525138855
Epoch 10: Gen Loss: 0.00917163398116827, Disc Loss: 0.3891550600528717
Epoch 10: Gen Loss: 0.008716350421309471, Disc Loss: 0.481600284576416
Epoch 10: Gen Loss: 0.008861927315592766, Disc Loss: 0.38798514008522034
Epoch 10: Gen Loss: 0.008717169985175133, Disc Loss: 0.4400402903556824
Epoch 10: Gen Loss: 0.008982707746326923, Disc Loss: 0.4337623119354248
Epoch 10: Gen Loss: 0.009350527077913284, Disc Loss: 0.4032137989997864
Epoch 10: Gen Loss: 0.010094130411744118, Disc Loss: 0.3538739085197449
Epoch 10: Gen Loss: 0.009086004458367825, Disc Loss: 0.46593424677848816
Epoch 10: Gen Loss: 0.009736405685544014, Disc Loss: 0.434373110

In [51]:
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

In [10]:
generator.load_state_dict(torch.load('generator.pth'))
discriminator.load_state_dict(torch.load('discriminator.pth'))
generator.to(device)
discriminator.to(device)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2)
    (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): LeakyReLU(negative_slope=0.2)
    (6): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.2)
    (8): AdaptiveAvgPool2d(output_size=1)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [15]:
class DatasetFromFolder(Dataset):
    def __init__(self, directory, input_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.directory = directory
        self.image_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.input_transform = input_transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert('RGB')

        if self.input_transform:
            img = self.input_transform(img)
        else:
            img = transforms.ToTensor()(img) 
        return img 
    
input_transform = transforms.Compose([
    transforms.Resize((64, 64)), 
    transforms.ToTensor(),
])

test_dataset = DatasetFromFolder(
    r"F:\super_resolution_project\lr_facial_img",
    input_transform=input_transform
)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [16]:
generator.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)

output_directory = r"F:\super_resolution_project\super_resolution_images"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

with torch.no_grad():
    for i, low_res in enumerate(test_loader):
        low_res = low_res.to(device)
        high_res_fake = generator(low_res)
        save_path = os.path.join(output_directory, f'high_resolution_fake_img_{i}.png')
        save_image(high_res_fake, save_path)
        print(f"Saved image to: {save_path}")

print("\n", f"Task Done")

Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_0.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_1.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_2.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_3.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_4.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_5.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_6.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_7.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_8.png
Saved image to: F:\super_resolution_project\super_resolution_images\high_resolution_fake_img_9.png
Saved imag